# Analytic solution of a two- and three-levels system

## Two level system

The analtic analytic ratios of the the level poputlations at equilbrium is presented and compared to the
numerical solution obtained by [Frigus](https://github.com/mherkazandjian/frigus).

### Analytic solution


$$
\mathbf{A} = 
\begin{bmatrix}
    0      & 0 \\
    A_{10} & 0 \\
\end{bmatrix}
$$

$$
\mathbf{B J_\nu} = 
\begin{bmatrix}
    0      & \frac{g_1}{g_0} A_{10} \\
    A_{10} & 0 \\
\end{bmatrix}
=
\begin{bmatrix}
    0      & B_{01} \\
    B_{10} & 0 \\
\end{bmatrix}
$$

$$
\mathbf{K} = 
\begin{bmatrix}
    0      & \frac{g_1}{g_0} K_{10} e^{-\frac{|E_{10}|}{k_b T_{\rm kin}}}\\
    K_{10} & 0 \\
\end{bmatrix}
=
\begin{bmatrix}
    0      & K_{01} \\
    K_{10} & 0 \\
\end{bmatrix}
$$

$$
\mathbf{n} = 
\begin{bmatrix}
    n_0 \\
    n_1 \\
\end{bmatrix}
$$


$$
\frac{d\mathbf{n}}{dt} = \mathbf{M} \cdot \mathbf{n} = \left[\left( \mathbf{A + B J_\nu + K}n_c \right)^T - (\mathbf{e}^T \cdot \mathbf{O}) \circ \mathbf{I}\right] \cdot \mathbf{n} \\
=
\begin{bmatrix}
    -(B_{01} + K_{01} n_c)
    & A_{10} + B_{10} + K_{10} n_c \\
    B_{01} + K_{01} n_c
    & -(A_{10} + B_{10} + K_{10} n_c)  \\
\end{bmatrix}
\cdot
\begin{bmatrix}
    n_0\\
    n_1\\
\end{bmatrix}
$$


The rate equations are

\begin{eqnarray}
\frac{dn_0}{dt} &=& -(B_{01} + K_{01}n_c) n_0 + (A_{10} + B_{10} + K_{10} n_c)n_1 \\
\frac{dn_1}{dt} &=& (B_{01} + K_{01}n_c) n_0 - (A_{10} + B_{10} + K_{10} n_c) n_1
\end{eqnarray}


At equilibrium, the ratio $n_1/n_0$ is:

$$                                                                                                                    \frac{n_1}{n_0} = \frac{B_{01} + K_{01}n_c}{A_{10} + B_{10} + K_{10}n_c} = \frac{M_{10}}{M_{01}}                 $$

### Comparison to the numerical solution obtained with Frigus 

In [6]:
# ..todo:: add the numerical solution here

### Three-level system

For a 3-levels system, the matrix formulation for the rate equations can be expressed as

$$
\mathbf{A} =
\begin{bmatrix}
    0      & 0      &  0  \\
    A_{10} & 0      &  0  \\
    A_{20} & A_{21} &  0  \\
\end{bmatrix}
$$                          

$$
\mathbf{B J_\nu} =
\begin{bmatrix}
    0      & \frac{g_1}{g_0} f_{10} A_{10} & \frac{g_2}{g_0} f_{20} A_{20} \\
    f_{10} A_{10} & 0                      & \frac{g_2}{g_1} f_{21} A_{21} \\
    f_{20} A_{20} & f_{21} A_{21}                 &   0                     \\
\end{bmatrix} =
\begin{bmatrix}
    0        & B_{01} &  B_{02} \\
    B_{10}   & 0      &  B_{12} \\
    B_{20}   & B_{21} &  0 \\
\end{bmatrix}
$$                                                                            

$$
\mathbf{K}=
\begin{bmatrix}
    0     & \frac{g_1}{g_0}K_{10}e^{-\frac{|E_{10}|}{k_b T_{kin}}} &  \frac{g_2}{g_0}K_{20}e^{-\frac{|E_{20}|}{k_b T_{kin}}} \\
    K_{10}& 0                                                      &  \frac{g_2}{g_1}K_{21}e^{-\frac{|E_{21}|}{k_b T_{kin}}} \\
    K_{20}& K_{20}                                                 &  0                                                      \\
\end{bmatrix} =
\begin{bmatrix}
    0      & K_{01} & K_{02}\\
    K_{10} & 0      & K_{12}\\
    K_{20} & K_{21} & 0     \\
\end{bmatrix};
\mathbf{n}=
\begin{bmatrix}
    n_0\\
    n_1\\
    n_2\\
\end{bmatrix}
$$


The rate equations are:
\begin{equation}
\begin{split}
\frac{dn_0}{dt} =& -(B_{01} + K_{01}n_c) n_0 -(B_{02} + K_{02}n_c) n_0 \\
                 & +(A_{10} + B_{10} + K_{10} n_c)n_1 + (A_{20} + B_{20} + K_{20} n_c)n_2\\
                =& -R_{01} n_0 - R_{02} n_0 \\
                 & + R_{10}n_1 + R_{20} n_2\\
\frac{dn_1}{dt} =& -(B_{12} + K_{12}n_c) n_1 - (A_{10} + B_{10} + K_{10} n_c)n_1  \\
                 & +(B_{01} + K_{01}n_c) n_0  + (A_{21} + B_{21} + K_{21} n_c)n_2\\
                =& -R_{12} n_1 - R_{10} n_1 \\
                 & + R_{01}n_0 + R_{21} n_2\\
\frac{dn_2}{dt} =& -(A_{21} + B_{21} + K_{21} n_c)n_2 -(A_{20} + B_{20} + K_{20} n_c)n_2\\
                 & +(B_{02} + K_{02}n_c) n_0  + (B_{12} + K_{12}n_c) n_1\\
                =& -R_{21} n_1 - R_{20} n_2 \\
                 & + R_{02}n_0 + R_{12} n_1\\
\end{split}
\end{equation}


where the terms in parenthesis have been grouped as $R_{ij}$ to facilitate the arithmetic
procedure of recovering the ratios of the level population at the equilibrium (as adopted
by [Draine 2010](https://press.princeton.edu/titles/9499.html)). Indeed, in such conditions, it can be easily found that
$n_2/n_1$ and $n_0/n_1$ are:
$$                                                                                                      
\frac{n_2}{n_1} = \frac{R_{10}R_{02}+R_{12}R_{01}+R_{12}R_{02}}{R_{21}R_{01}+R_{21}R_{02}+R_{01}R_{20}} 
$$
$$                                                                                                      
\frac{n_0}{n_1} = \frac{R_{10}R_{20}+R_{10}R_{21}+R_{12}R_{20}}{R_{01}R_{20}+R_{01}R_{21}+R_{21}R_{02}} 
$$


### Comparison to the numerical solution obtained with Frigus 

In [7]:
# ..todo:: add the numerical solution here